In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [4]:
train = pd.read_csv("input/sales_train.csv")
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.000,1.000
1,03.01.2013,0,25,2552,899.000,1.000
2,05.01.2013,0,25,2552,899.000,-1.000
3,06.01.2013,0,25,2554,1709.050,1.000
4,15.01.2013,0,25,2555,1099.000,1.000


In [11]:
tests = pd.read_csv("input/test.csv")
tests.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
train.groupby(["item_id"]).item_price.agg([len])[:5]

,len
item_id,
0,1.000
1,6.000
2,2.000
3,2.000
4,1.000


In [6]:
train[(train["item_id"] ==22140)].sort_values(by="shop_id")

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
44841,26.01.2013,0,0,22140,312.000,1.000
2705819,17.06.2015,29,2,22140,199.000,1.000
1733710,27.06.2014,17,4,22140,299.000,1.000
164317,24.02.2013,1,4,22140,499.000,1.000
374147,12.04.2013,3,5,22140,499.000,1.000
1954865,27.09.2014,20,5,22140,217.500,1.000
876144,27.09.2013,8,5,22140,499.000,1.000
2618782,09.05.2015,28,5,22140,299.000,1.000
1656244,16.05.2014,16,5,22140,239.360,1.000
1649255,06.05.2014,16,6,22140,299.000,1.000


I think I should add an item_price column. It should be imputed based on the date, shop_id, and item_id

In [7]:
train = pd.read_csv("input/sales_train.csv")
test = pd.read_csv("input/test.csv")
items = pd.read_csv("input/items.csv")
itemsCat = pd.read_csv("input/item_categories.csv")
shops = pd.read_csv("input/shops.csv")

import math
def get_second(strings):
    if isinstance(strings, str):
        temp = strings.split(" - ")
        if len(temp) > 1:
            return temp[1].strip()
        return "NA"
    return "NA"
def get_first(strings):
    if isinstance(strings, str):
        temp = strings.split(" - ")
        return temp[0].strip()
    return "NA"

train = train.merge(items, left_on='item_id', right_on='item_id', how='outer')
train = train.merge(itemsCat, left_on='item_category_id', right_on='item_category_id', how='outer')
train = train.merge(shops, left_on='shop_id', right_on='shop_id', how='outer')
train.drop(["item_id","item_category_id","shop_id"],axis=1, errors="ignore",inplace=True)
train["cat1"] = train.item_category_name.map(lambda x: x.split(" - ")[0].strip())
train["cat2"] = train.item_category_name.map(lambda x: get_second(x))

test = test.merge(items, left_on='item_id', right_on='item_id', how='outer')
test = test.merge(itemsCat, left_on='item_category_id', right_on='item_category_id', how='outer')
test = test.merge(shops, left_on='shop_id', right_on='shop_id', how='outer')
test.drop(["item_id","item_category_id","shop_id"],axis=1, errors="ignore",inplace=True)
test["cat1"] = test.item_category_name.map(lambda x: get_first(x))
test["cat2"] = test.item_category_name.map(lambda x: get_second(x))

In [8]:
train

,date,date_block_num,item_price,item_cnt_day,item_name,item_category_name,shop_name,cat1,cat2
0,02.01.2013,0.000,999.000,1.000,ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
1,26.04.2013,3.000,150.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
2,26.06.2013,5.000,199.500,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
3,20.07.2013,6.000,199.500,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
4,14.09.2013,8.000,299.000,2.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
5,21.10.2013,9.000,299.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
6,02.11.2013,10.000,299.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
7,06.01.2014,12.000,299.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
8,30.03.2014,14.000,299.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
9,20.04.2014,15.000,199.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray


In [9]:
test

,ID,item_name,item_category_name,shop_name,cat1,cat2
0,0.000,"NHL 15 [PS3, русские субтитры]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
1,2.000,"Need for Speed Rivals (Essentials) [PS3, русск...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
2,15.000,"Minecraft. Playstation 3 Edition [PS3, русская...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
3,21.000,"NBA 2K16 [PS3, русская документация]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
4,23.000,"Plants vs. Zombies Garden Warfare [PS3, русска...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
5,56.000,"FIFA 15 [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
6,58.000,"FIFA 16 [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
7,62.000,"FIFA Street (Essentials) [PS3, английская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
8,69.000,"God of War 3 (Essentials) [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
9,81.000,"Far Cry 4 [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3


In [13]:
test = pd.read_csv("test.csv")
test = test.merge(items, left_on='item_id', right_on='item_id', how='outer')

In [21]:
#I think I can drop item_category_name. We'll see how well it fares w/ and w/o it
train.head()

,date,date_block_num,item_price,item_cnt_day,item_name,item_category_name,shop_name,cat1,cat2
0,02.01.2013,0.000,999.000,1.000,ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
1,26.04.2013,3.000,150.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
2,26.06.2013,5.000,199.500,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
3,20.07.2013,6.000,199.500,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray
4,14.09.2013,8.000,299.000,2.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray


In [9]:
#I am predicting for november 2015
test.head()

,ID,item_name,item_category_name,shop_name,cat1,cat2
0,0.000,"NHL 15 [PS3, русские субтитры]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
1,2.000,"Need for Speed Rivals (Essentials) [PS3, русск...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
2,15.000,"Minecraft. Playstation 3 Edition [PS3, русская...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
3,21.000,"NBA 2K16 [PS3, русская документация]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
4,23.000,"Plants vs. Zombies Garden Warfare [PS3, русска...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
5,56.000,"FIFA 15 [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
6,58.000,"FIFA 16 [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
7,62.000,"FIFA Street (Essentials) [PS3, английская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
8,69.000,"God of War 3 (Essentials) [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3
9,81.000,"Far Cry 4 [PS3, русская версия]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3


Things to investigate:
 - the annual trends
 - the monthly trends
 - the day of week trends
 - anything major that might've happened in russia in november
     - local news?

Ideas for dates:
 - is_weekend
 - is_holiday (are there russian holidays in november?)
 
The problem is that there isn't a date in the test dataset.
ideas:
 - have 2 models. One specifically for November, one for the whole year
 - November has a weight of 70% and the year one will have a weight of 30%

In [23]:
def get_year(strings):
    if isinstance(strings, str):
        temp = strings.split(".")
        if len(temp) > 2:
            return temp[2]
        return -1
    return -1
def get_month(strings):
    if isinstance(strings, str):
        temp = strings.split(".")
        if len(temp) > 1:
            return temp[1]
        return -1
    return -1

train["year"] = train.date.map(lambda x: get_year(x))
train["month"] = train.date.map(lambda x: get_month(x))
train = train[train["month"] != -1]
train.head()

,date,date_block_num,item_price,item_cnt_day,item_name,item_category_name,shop_name,cat1,cat2,year,month
0,02.01.2013,0.000,999.000,1.000,ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,01
1,26.04.2013,3.000,150.000,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,04
2,26.06.2013,5.000,199.500,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,06
3,20.07.2013,6.000,199.500,1.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,07
4,14.09.2013,8.000,299.000,2.000,2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,09


In [31]:
train.drop(["date","date_block_num","item_category_name"],axis=1,inplace=True,errors="ignore")
test.drop(["item_category_name"],axis=1,inplace=True,errors="ignore")
test["year"] = 2015
test["month"] = 11
train.head()

,item_price,item_cnt_day,item_name,shop_name,cat1,cat2,year,month
0,999.000,1.000,ЯВЛЕНИЕ 2012 (BD),"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,01
1,150.000,1.000,2012 (BD),"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,04
2,199.500,1.000,2012 (BD),"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,06
3,199.500,1.000,2012 (BD),"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,07
4,299.000,2.000,2012 (BD),"Ярославль ТЦ ""Альтаир""",Кино,Blu-Ray,2013,09


In [32]:
test.head()

,ID,item_name,shop_name,cat1,cat2,year,month
0,0.000,"NHL 15 [PS3, русские субтитры]","Вологда ТРЦ ""Мармелад""",Игры,PS3,2015,11
1,2.000,"Need for Speed Rivals (Essentials) [PS3, русск...","Вологда ТРЦ ""Мармелад""",Игры,PS3,2015,11
2,15.000,"Minecraft. Playstation 3 Edition [PS3, русская...","Вологда ТРЦ ""Мармелад""",Игры,PS3,2015,11
3,21.000,"NBA 2K16 [PS3, русская документация]","Вологда ТРЦ ""Мармелад""",Игры,PS3,2015,11
4,23.000,"Plants vs. Zombies Garden Warfare [PS3, русска...","Вологда ТРЦ ""Мармелад""",Игры,PS3,2015,11


In [33]:
train.to_pickle("ftrain")
test.to_pickle("ftest")